# Malware Detection in Network Traffic

## Random Forest Classification

Run the quickstart_ml_eda.ipynb to obtain the preprocessed data for this analysis.

This tutorial is based on the following work:

https://github.com/aruberts/tutorials/blob/main/pyspark/spark_feature_engineering.ipynb

https://www.youtube.com/watch?v=TlXqsL4ysB0&t=1322s

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pyspark==3.5.1

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier

In [ ]:
spark = (
    SparkSession.builder.appName("iot")
    .getOrCreate()
)

## Load preprocessed Parquet data

Load data preprocessed in the previous tutorial and while loading create a new variable "is_bad" with the numerical value 1 for malicious traffic and 0 for benign.

In [ ]:
df_fe = spark.read.parquet("/content/drive/MyDrive/NetworkMalwareData/processed.pq").withColumn(
    "is_bad", F.when(F.col("label") != "Benign", 1).otherwise(0)
)
df_fe.show(5)

## Preprocessing

Select numerical and categorical features for analysis:

In [ ]:
numerical_features = [
    "duration",
    "orig_bytes",
    "resp_bytes",
    "orig_pkts",
    "orig_ip_bytes",
    "resp_pkts",
    "resp_ip_bytes",
]
categorical_features = ["proto", "service", "conn_state", "history"]
categorical_features_indexed = [c + "_index" for c in categorical_features]

input_features = numerical_features + categorical_features_indexed

### Remove rare categories
Count unique entries for each categorical variable and delete rare categories (less than 100 unique entries).

In [ ]:
df_fe.select([F.count_distinct(c) for c in categorical_features]).show()

In [ ]:
categorical_valid_values = {}

for c in categorical_features:
    # Find frequent values
    categorical_valid_values[c] = (
        df_fe.groupby(c)
        .count()
        .filter(F.col("count") > 100)
        .select(c)
        .toPandas()
        .values.ravel()
    )

    df_fe = df_fe.withColumn(
        c,
        F.when(F.col(c).isin(list(categorical_valid_values[c])), F.col(c)).otherwise(
            F.lit("Other").alias(c)
        ),
    )

Confirm results:

In [ ]:
df_fe.select([F.count_distinct(c) for c in categorical_features]).show()

## Train/Test Split
The Train-test split needs to be done using the source IP address, otherwise we risk leaking data. First we count and order the malicious IP addresses:

In [ ]:
df_fe.groupby("source_ip").agg(F.sum(F.col("is_bad")).alias("bad_sum")).orderBy("bad_sum", ascending=False).show(5)

Now we can select 80% of the non-malicious IP addresses for the training set and the rest for the test set by creating a new variable is_train. Then we can select one of the top malicious IP addresses for the training data and the other IP address for the test data.

In [ ]:
# Training non-malicious IPs (80%)
train_ips = (
    df_fe.where(
        ~F.col("source_ip").isin(["192.168.100.103", "192.168.2.5", "192.168.2.1"])
    )
    .select(F.col("source_ip"), F.lit(1).alias("is_train"))
    .dropDuplicates()
    .sample(0.8)
)


df_fe = df_fe.join(train_ips, "source_ip", "left")

# Add 1 malicious IP to training and testing data
df_train = df_fe.where((F.col("is_train") == 1) | (F.col("source_ip") == "192.168.100.103"))
df_test = df_fe.where((F.col("is_train") != 1) | (F.col("source_ip") == "192.168.2.5"))

## Pipeline

Create pipeline for machine learning: one-hot encode categorical features, assemble features in a single vector, scale features, run random forest classification.

In [ ]:
ind = StringIndexer(inputCols=categorical_features, outputCols=categorical_features_indexed, handleInvalid='skip')
va = VectorAssembler(inputCols=input_features, outputCol="features", handleInvalid='skip' )
ss = StandardScaler(inputCol="features", outputCol="features_scaled")
rf = RandomForestClassifier(featuresCol="features_scaled", labelCol="is_bad", numTrees=100)

pipeline = Pipeline(stages=[ind, va, ss, rf])

## Fit and Predict

Fit pipeline to training data and transform using test data.

In [ ]:
pipeline = pipeline.fit(df_train)
test_preds = pipeline.transform(df_test)

## Evaluate

Print classification metrics to better understand results (e.g. ROC/PR area under the curve).

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

roc = BinaryClassificationEvaluator(labelCol="is_bad", metricName="areaUnderROC")
print("ROC AUC", roc.evaluate(test_preds))

pr = BinaryClassificationEvaluator(labelCol="is_bad", metricName="areaUnderPR")
print("PR AUC", pr.evaluate(test_preds))

Create Pandas dataframe of importance metric for each feature:

In [ ]:
import pandas as pd

pd.DataFrame(
    {
        "importance": list(pipeline.stages[-1].featureImportances),
        "feature": pipeline.stages[-3].getInputCols(),
    }
).sort_values("importance", ascending=False)

## Export

Save pipeline (either only random forest classifier or full pipeline):

In [ ]:
pipeline.stages[-1].write().overwrite().save("rf_basic")

In [ ]:
pipeline.write().overwrite().save("pipeline_basic")